# OpenUSD Tooling for  MaterialX

## Introduction

In this book we will look at general tools to take MaterialX and convert it to USD.
Namely we are interested in `UsdMtlx` and utilities from the `Usd Toolset`. Note that `usdview` is also a useful basic tool for inspecting and visualizing USD files. 

At time of writing the the Python package for Usd from PyPi does not include these tools.

While it is possible to download the repository and build it yourself, using prebuilt binaries is less trouble. For instance NVIDIA has pre-built binaries for Linux and Windows available <a href="https://developer.nvidia.com/usd" target="_blank">here</a> 
( A download of one of these binaries is used for this notebook. If you are working with a specific build for an integration such as a DCC such as Maya or Houdini, that can be used instead. )

For these builds, ensure that the correct Python and library paths are set up. See <a href="https://openusd.org/release/tut_usd_tutorials.html" target="_blank">the USD tutorials</a> for more information. Instead of setting the `PYTHONPATH` it is possible to install the Python libraries in your Python (virtual) environment.

In [14]:
import os

# Print path reference which should include the Python path of the USD build
print('PYTHON PATH :', '"' + os.getenv('PYTHONPATH') + '"')

PYTHON PATH : "D:\Downloads\usd.py310.windows-x86_64.usdview.release@0.23.08-tc.18+v23.08.10b62439\lib\python;"


### Setup OpenUSD and MaterialX

In the code below we import both the MaterialX and Usd core library

In [15]:
from pxr import Usd
import MaterialX as mx

# For Markdown output display
from IPython.display import display_markdown

major, minor, build = Usd.GetVersion() 
print('Using Usd Version:', str(major) + "." + str(minor) + "." + str(build))
print('Using MaterialX Version:', mx.getVersionString())


Using Usd Version: 0.23.8
Using MaterialX Version: 1.38.9


For the purposes of examining the output, we add a couple of utility functions to print file directly or as XML.

In [16]:
def printFileRaw(file):
    f = open(file, 'r')
    print(f.read())
    f.close()

def printFileAsXML(file, title, fmt):
    f = open(file, 'r')
    text = '<details><summary>' + title + '</summary>\n\n' + '```' + fmt + '\n' + f.read() + '```\n' + '</details>\n' 
    display_markdown(text , raw=True)            
    f.close()

### Checking Shader Registry for MaterialX Nodes

Next we want to check that the node definitions for MaterialX are registered in the shader definition registry 
(<a href="https://openusd.org/dev/api/class_sdr_registry.html" target="_blank">Sdr</a>)

By default USD will not discover the MaterialX definition libraries. It is thus necessary to explicitly specify the path to these libraries. We use the `PXR_PLUGINPATH_NAME` environment variable to do this by setting it to the `libraries` folder of the MaterialX distribution. 

To test that the path is properly registered `Sdr.Registry().GetSearchURIs()` can be used.

Another check that can be performed is to find the available node "source types" using `Sdr.Registry().GetAllNodeSourceTypes()`, for which MaterialX is `mtlx`.

The "convention" used for MaterialX is to prefix node definition identifiers with `ND_`. We can use this to check that the MaterialX nodes are available in the shader registry.

In [17]:
# Import Sdr module
from pxr import Sdr

# Check that we have MaterialX definitions
usdmtlxPath = os.environ.get('PXR_MTLX_STDLIB_SEARCH_PATHS')
print('Pixar MaterialX Library Path:' + '"' + usdmtlxPath + '"')

sourceTypes = Sdr.Registry().GetAllNodeSourceTypes()
if 'mtlx' in sourceTypes:
    print('Sdr MaterialX source type \"mtlx\" is registered')
else:
    print('Sdr MaterialX source type \"mtlx\" is not registered')

print('Sdr Search Paths:')
for uri in Sdr.Registry().GetSearchURIs():
    print("- \"" + uri + "\"")

# Get the names of definitions in Sdr
documentContents = ''
if Sdr.Registry():
    for node in Sdr.Registry().GetNodeNames():        
        documentContents = documentContents + ('- %s\n' % node)

text = '<details><summary>Shader Registry Nodes</summary>\n\n' + '```xml\n' + documentContents + '```\n' + '</details>\n' 
display_markdown(text , raw=True)        


Pixar MaterialX Library Path:"D:\Downloads\usd.py310.windows-x86_64.usdview.release@0.23.08-tc.18+v23.08.10b62439\libraries"
Sdr MaterialX source type "mtlx" is registered
Sdr Search Paths:
- "d:/Downloads/usd.py310.windows-x86_64.usdview.release@0.23.08-tc.18+v23.08.10b62439/lib/usd/usdHydra/resources/shaders"
- "D:\Downloads\usd.py310.windows-x86_64.usdview.release@0.23.08-tc.18+v23.08.10b62439\libraries"
- "W:/b4a5145e5e524e02/_install/py310.windows-x86_64.usdview.release/libraries"
- "d:/Downloads/usd.py310.windows-x86_64.usdview.release@0.23.08-tc.18+v23.08.10b62439/plugin/usd/usdShaders/resources/shaders"


<details><summary>Shader Registry Nodes</summary>

```xml
- ND_power_color3FA
- ND_invertmatrix_matrix44
- HwPtexTexture
- ND_range_color4FA
- ND_UsdPrimvarReader_vector3
- ND_ramp4_vector2
- ND_max_color3FA
- HwFieldReader_float
- ND_conductor_bsdf
- ND_divide_color3FA
- ND_transformmatrix_vector2M3
- HwUvTexture
- HwFieldReader
- ND_contrast_color3
- ND_lama_sheen
- ND_lama_dielectric
- HwFieldReader_float2
- ND_image_color3
- ND_UsdPrimvarReader_string
- ND_invert_color3FA
- ND_absval_float
- ND_gltf_image_vector3_vector3_1_0
- ND_divide_color4FA
- ND_power_vector3FA
- HwFieldReader_float3
- ND_gltf_image_color4_color4_1_0
- ND_smoothstep_vector2FA
- ND_tiledimage_float
- HwPrimvar
- ND_noise2d_vector3FA
- ND_colorcorrect_color4
- ND_disney_brdf_2012_surface
- ND_standard_surface_surfaceshader
- ND_disney_bsdf_2015_surface
- ND_sin_vector2
- ND_gltf_pbr_surfaceshader
- ND_ifgreatereq_color3I
- ND_gltf_colorimage
- ND_add_vector3FA
- ND_invert_vector3
- ND_gltf_image_color3_color3_1_0
- ND_gltf_image_float_float_1_0
- ND_standard_surface_to_gltf_pbr
- ND_gltf_normalmap_vector3_1_0
- ND_glossiness_anisotropy
- ND_thin_surface
- ND_gltf_iridescence_thickness_float_1_0
- ND_UsdPreviewSurface_surfaceshader
- ND_rgbtohsv_color4
- ND_UsdUVTexture
- ND_add_color4
- ND_anisotropic_vdf
- ND_lama_layer_bsdf
- ND_extract_vector2
- ND_UsdPrimvarReader_integer
- ND_noise3d_color3FA
- ND_tan_vector3
- ND_UsdPrimvarReader_boolean
- ND_position_vector3
- ND_lama_translucent
- ND_geompropvalue_color3
- ND_extract_vector4
- ND_swizzle_color4_float
- ND_UsdTransform2d
- ND_remap_vector4
- ND_UsdPrimvarReader_float
- ND_UsdPrimvarReader_vector2
- ND_uniform_edf
- ND_sheen_bsdf
- ND_tangent_vector3
- ND_burley_diffuse_bsdf
- ND_noise2d_color3
- ND_UsdPrimvarReader_vector4
- ND_invert_float
- ND_ifequal_color4B
- ND_invert_vector4FA
- ND_range_vector2FA
- ND_lama_mix_bsdf
- ND_lama_add_bsdf
- ND_lama_add_edf
- ND_noise3d_color4FA
- UsdPreviewSurface
- ND_generalized_schlick_bsdf
- ND_lama_conductor
- ND_noise2d_vector2FA
- ND_swizzle_vector2_vector4
- ND_lama_diffuse
- ND_lama_emission
- ND_lama_mix_edf
- ND_lama_sss
- ND_triplanarprojection_color4
- ND_dotproduct_vector4
- ND_conical_edf
- ND_plus_color4
- ND_standard_surface_to_UsdPreviewSurface
- ND_point_light
- ND_divide_vector4
- ND_constant_vector3
- ND_acos_vector2
- ND_directional_light
- ND_layer_bsdf
- ND_noise3d_vector4FA
- ND_dielectric_bsdf
- ND_spot_light
- ND_ifequal_color4I
- ND_oren_nayar_diffuse_bsdf
- ND_translucent_bsdf
- ND_mix_vdf
- ND_splitlr_color3
- ND_subsurface_bsdf
- ND_thin_film_bsdf
- ND_convert_float_surfaceshader
- ND_cos_vector4
- ND_transformpoint_vector3
- ND_dot_vector4
- ND_measured_edf
- ND_generalized_schlick_edf
- ND_swizzle_float_color4
- ND_absorption_vdf
- ND_geomcolor_color3
- ND_normalize_vector3
- ND_surface
- ND_volume
- ND_arrayappend_color3array_color3array
- ND_light
- ND_displacement_float
- ND_displacement_vector3
- ND_normal_vector3
- ND_layer_vdf
- ND_add_vdf
- ND_mix_bsdf
- ND_dot_float
- ND_mix_edf
- ND_screen_color4
- ND_add_bsdf
- ND_noise3d_vector3FA
- ND_swizzle_vector3_color3
- ND_add_edf
- ND_multiply_bsdfC
- ND_multiply_color3FA
- ND_absval_vector2
- ND_separate3_vector3
- ND_multiply_bsdfF
- ND_surface_unlit
- ND_noise3d_vector4
- ND_multiply_edfC
- ND_ramp4_vector3
- ND_min_vector4
- ND_multiply_edfF
- ND_multiply_vdfC
- ND_clamp_color4
- ND_luminance_color4
- ND_multiply_vdfF
- ND_fractal3d_vector3
- ND_roughness_anisotropy
- ND_multiply_matrix44
- ND_triplanarprojection_vector4
- ND_subtract_vector3FA
- ND_floor_float
- ND_roughness_dual
- ND_min_vector2FA
- ND_blackbody
- ND_fractal3d_color3FA
- ND_artistic_ior
- ND_splittb_color4
- ND_surfacematerial
- ND_contrast_vector4
- ND_min_color3FA
- ND_volumematerial
- ND_add_color3FA
- ND_geompropvalue_color4
- ND_image_float
- ND_image_color4
- ND_power_color3
- ND_atan2_vector2
- ND_constant_matrix44
- ND_texcoord_vector3
- ND_image_vector2
- ND_ceil_float
- ND_image_vector3
- ND_geompropvalue_vector4
- ND_image_vector4
- ND_multiply_vector3FA
- ND_tiledimage_color3
- ND_tiledimage_color4
- ND_tiledimage_vector2
- ND_constant_float
- ND_tiledimage_vector3
- ND_tiledimage_vector4
- ND_ln_float
- ND_fractal3d_vector4
- ND_triplanarprojection_float
- ND_triplanarprojection_color3
- ND_triplanarprojection_vector2
- ND_smoothstep_color3
- ND_triplanarprojection_vector3
- ND_constant_color3
- ND_constant_color4
- ND_constant_vector2
- ND_dot_surfaceshader
- ND_constant_vector4
- ND_subtract_vector3
- ND_constant_boolean
- ND_range_vector3
- UsdTransform2d
- ND_extract_vector3
- ND_min_color3
- ND_constant_integer
- ND_constant_matrix33
- ND_arrayappend_vector2_vector2array
- ND_cos_vector2
- ND_constant_string
- ND_transpose_matrix33
- ND_ifgreatereq_vector4
- ND_constant_filename
- ND_ramplr_float
- ND_ramplr_color3
- ND_ifgreatereq_vector2
- ND_ramplr_color4
- ND_subtract_vector4
- ND_ramplr_vector2
- ND_ramplr_vector3
- ND_clamp_vector3
- ND_geompropvalue_float
- ND_arrayappend_vector4_vector4array
- ND_convert_float_vector4
- ND_ramplr_vector4
- ND_ramptb_float
- ND_ramptb_color3
- ND_ramptb_color4
- ND_fractal3d_vector3FA
- ND_difference_color3
- ND_ramptb_vector2
- ND_ramptb_vector3
- ND_ramptb_vector4
- ND_swizzle_float_color3
- ND_ramp4_float
- ND_ramp4_color3
- ND_geompropvalue_string
- ND_ramp4_color4
- ND_swizzle_color3_vector2
- ND_multiply_vector4
- ND_ramp4_vector4
- ND_splitlr_float
- ND_inside_float
- ND_splitlr_color4
- ND_splitlr_vector2
- ND_splitlr_vector3
- ND_splitlr_vector4
- ND_splittb_float
- ND_curveadjust_color4
- ND_plus_color3
- RectLight
- ND_splittb_color3
- ND_mix_displacementshader
- ND_splittb_vector2
- ND_atan2_float
- ND_invert_vector2FA
- ND_dotproduct_vector3
- ND_splittb_vector3
- ND_splittb_vector4
- ND_premult_color4
- ND_remap_vector2
- ND_noise2d_float
- ND_noise2d_color3FA
- ND_overlay_float
- ND_noise2d_color4
- ND_rotate3d_vector3
- ND_noise2d_vector2
- ND_noise2d_vector3
- ND_power_vector4
- ND_acos_float
- ND_ifequal_floatI
- ND_noise2d_vector4
- ND_determinant_matrix33
- ND_multiply_vector2FA
- ND_noise2d_color4FA
- ND_crossproduct_vector3
- ND_noise2d_vector4FA
- ND_noise3d_float
- ND_subtract_color3FA
- ND_noise3d_color3
- ND_noise3d_color4
- ND_noise3d_vector2
- ND_clamp_vector2
- ND_noise3d_vector3
- ND_noise3d_vector2FA
- ND_fractal3d_float
- ND_fractal3d_color3
- ND_fractal3d_color4
- ND_worleynoise3d_vector3
- ND_fractal3d_vector2
- ND_curveadjust_vector3
- ND_fractal3d_color4FA
- ND_fractal3d_vector2FA
- ND_fractal3d_vector4FA
- ND_swizzle_color3_float
- ND_convert_boolean_float
- ND_cellnoise2d_float
- ND_normalmap
- ND_cellnoise3d_float
- ND_normalize_vector2
- ND_worleynoise2d_float
- ND_worleynoise2d_vector2
- ND_remap_vector3FA
- ND_worleynoise2d_vector3
- ND_subtract_matrix33FA
- ND_clamp_vector3FA
- ND_worleynoise3d_float
- ND_arrayappend_color4array_color4array
- ND_dot_displacementshader
- ND_worleynoise3d_vector2
- ND_modulo_vector2FA
- ND_overlay_color4
- ND_unifiednoise2d_float
- ND_unifiednoise3d_float
- ND_max_vector4FA
- ND_separate3_color3
- ND_swizzle_vector4_color3
- ND_min_vector2
- ND_remap_color3
- ND_bitangent_vector3
- ND_swizzle_vector4_vector4
- ND_texcoord_vector2
- ND_ifequal_floatB
- ND_geomcolor_float
- ND_floor_vector4
- ND_saturate_color4
- ND_geomcolor_color4
- ND_subtract_color4FA
- ND_combine2_vector2
- ND_remap_color4
- ND_geompropvalue_integer
- ND_geompropvalue_boolean
- ND_contrast_color3FA
- ND_geompropvalue_vector2
- DistantLight
- ND_subtract_matrix44
- ND_geompropvalue_vector3
- ND_bump_vector3
- ND_ambientocclusion_float
- ND_divide_matrix44
- ND_frame_float
- ND_time_float
- ND_add_float
- ND_add_color3
- ND_add_vector2
- ND_transformmatrix_vector4
- ND_add_vector3
- ND_burn_float
- ND_add_vector4
- ND_add_matrix33
- ND_add_matrix44
- ND_add_color4FA
- ND_add_vector2FA
- ND_add_vector4FA
- ND_tan_vector2
- ND_min_color4
- ND_ifequal_vector2I
- ND_arrayappend_integerarray_integerarray
- ND_add_matrix33FA
- ND_add_matrix44FA
- ND_subtract_float
- ND_subtract_color3
- ND_contrast_float
- ND_subtract_color4
- ND_rotate2d_vector2
- ND_ifequal_vector2B
- ND_ifgreater_vector4
- ND_subtract_vector2
- ND_invert_vector2
- ND_subtract_matrix33
- ND_convert_vector4_vector3
- ND_subtract_vector2FA
- ND_subtract_vector4FA
- ND_subtract_matrix44FA
- ND_multiply_float
- ND_exp_vector3
- ND_multiply_color3
- ND_absval_color4
- ND_invertmatrix_matrix33
- ND_multiply_color4
- ND_multiply_vector2
- ND_multiply_vector3
- ND_multiply_matrix33
- ND_mix_vector4_vector4
- ND_multiply_color4FA
- ND_magnitude_vector3
- ND_multiply_vector4FA
- ND_divide_float
- ND_divide_color3
- ND_divide_color4
- ND_remap_float
- ND_divide_vector2
- ND_divide_vector3
- ND_divide_matrix33
- ND_divide_vector2FA
- ND_divide_vector3FA
- ND_divide_vector4FA
- ND_remap_vector4FA
- ND_modulo_float
- ND_modulo_color3
- ND_modulo_color4
- ND_modulo_vector2
- ND_unpremult_color4
- ND_modulo_vector3
- ND_sin_vector3
- ND_modulo_vector4
- ND_range_vector3FA
- ND_modulo_color3FA
- ND_modulo_color4FA
- ND_modulo_vector3FA
- ND_modulo_vector4FA
- ND_invert_color3
- ND_invert_color4
- ND_ceil_vector3
- ND_invert_vector4
- ND_range_vector2
- ND_burn_color3
- ND_arrayappend_floatarray_floatarray
- ND_invert_color4FA
- ND_clamp_float
- ND_invert_vector3FA
- ND_sign_float
- ND_swizzle_vector2_vector2
- ND_ifequal_vector2
- ND_absval_color3
- ND_absval_vector3
- ND_range_vector4
- ND_absval_vector4
- ND_floor_color3
- ND_floor_color4
- ND_floor_vector2
- ND_floor_vector3
- ND_minus_color3
- ND_ceil_color3
- ND_ifgreatereq_vector4I
- ND_ceil_color4
- ND_ceil_vector2
- ND_ceil_vector4
- ND_power_float
- ND_transpose_matrix44
- ND_power_color4
- ND_power_vector2
- ND_max_vector4
- ND_power_vector3
- ND_convert_color4_surfaceshader
- ND_power_color4FA
- ND_power_vector2FA
- ND_min_vector4FA
- ND_power_vector4FA
- ND_sin_float
- ND_arrayappend_vector3array_vector3array
- ND_cos_float
- ND_tan_float
- ND_asin_float
- ND_asin_vector2
- ND_cos_vector3
- ND_clamp_vector4FA
- ND_range_color3
- ND_switch_vector3
- ND_asin_vector3
- ND_ifgreater_color4I
- ND_acos_vector3
- ND_atan2_vector3
- ND_sin_vector4
- ND_tan_vector4
- ND_max_float
- ND_asin_vector4
- ND_smoothstep_color4
- ND_acos_vector4
- ND_mask_color4
- ND_atan2_vector4
- ND_sqrt_float
- ND_exp_float
- ND_place2d_vector2
- ND_sqrt_vector2
- ND_curveadjust_float
- ND_ln_vector2
- ND_clamp_vector4
- ND_exp_vector2
- ND_sqrt_vector3
- ND_ln_vector3
- ND_dot_lightshader
- ND_sqrt_vector4
- ND_hsvtorgb_color3
- ND_swizzle_vector3_vector4
- ND_contrast_vector4FA
- ND_ln_vector4
- ND_exp_vector4
- ND_sign_color3
- ND_sign_color4
- ND_sign_vector2
- GeometryLight
- ND_sign_vector3
- ND_sign_vector4
- ND_mix_float
- ND_minus_color4
- ND_clamp_color3
- ND_switch_floatI
- ND_clamp_color3FA
- ND_clamp_color4FA
- ND_curveadjust_color3
- ND_convert_color3_surfaceshader
- ND_ifgreatereq_vector3
- ND_clamp_vector2FA
- ND_convert_float_color4
- ND_min_float
- ND_min_vector3
- ND_min_color4FA
- ND_min_vector3FA
- ND_max_color3
- ND_burn_color4
- ND_max_color4
- UsdPrimvarReader_string
- ND_max_vector2
- ND_convert_float_vector2
- ND_arrayappend_color3_color3array
- ND_max_vector3
- ND_max_color4FA
- ND_blur_vector3
- ND_max_vector2FA
- ND_max_vector3FA
- ND_normalize_vector4
- ND_magnitude_vector2
- ND_magnitude_vector4
- ND_dotproduct_vector2
- ND_transformvector_vector3
- UsdPrimvarReader_float2
- ND_transformnormal_vector3
- ND_transformmatrix_vector3
- ND_transformmatrix_vector3M4
- ND_smoothstep_vector2
- ND_determinant_matrix44
- ND_arrayappend_integer_integerarray
- ND_arrayappend_float_floatarray
- ND_arrayappend_color4_color4array
- ND_ifgreater_float
- ND_arrayappend_vector2array_vector2array
- ND_arrayappend_vector3_vector3array
- ND_dot_integer
- ND_arrayappend_vector4array_vector4array
- ND_arrayappend_string_stringarray
- ND_smoothstep_color3FA
- ND_arrayappend_stringarray_stringarray
- ND_remap_vector3
- ND_convert_integer_float
- ND_remap_color3FA
- ND_remap_color4FA
- ND_remap_vector2FA
- ND_mix_vector3_vector3
- ND_smoothstep_float
- UsdPrimvarReader_float4
- ND_smoothstep_vector3
- ND_smoothstep_vector4
- ND_smoothstep_color4FA
- ND_smoothstep_vector3FA
- ND_smoothstep_vector4FA
- ND_hsvtorgb_color4
- ND_switch_vector3I
- ND_curveadjust_vector2
- ND_range_color3FA
- ND_curveadjust_vector4
- ND_ifgreatereq_vector3I
- ND_luminance_color3
- ND_hsvadjust_color3
- ND_rgbtohsv_color3
- ND_mix_volumeshader
- ND_contrast_color4
- ND_contrast_vector2
- ND_swizzle_vector2_float
- ND_contrast_vector3
- ND_contrast_color4FA
- ND_contrast_vector2FA
- ND_contrast_vector3FA
- ND_range_float
- ND_range_color4
- ND_range_vector4FA
- ND_hsvadjust_color4
- ND_mix_color3_color3
- ND_saturate_color3
- ND_colorcorrect_color3
- ND_plus_float
- ND_blur_vector4
- ND_minus_float
- ND_difference_float
- ND_difference_color4
- ND_dodge_float
- ND_dodge_color3
- ND_swizzle_color4_color3
- ND_ifequal_vector4
- ND_dodge_color4
- ND_switch_vector2
- ND_screen_float
- ND_screen_color3
- ND_over_color4
- ND_overlay_color3
- ND_disjointover_color4
- ND_in_color4
- ND_matte_color4
- ND_out_color4
- ND_inside_color3
- ND_inside_color4
- ND_outside_float
- ND_outside_color3
- ND_mix_surfaceshader
- ND_outside_color4
- ND_mix_color3
- ND_mix_color4
- ND_mix_color4_color4
- ND_separate4_color4
- ND_mix_vector2
- ND_mix_vector2_vector2
- ND_mix_vector3
- ND_mix_vector4
- ND_ifgreater_color4
- ND_ifgreater_color3
- ND_ifgreater_floatI
- ND_ifgreater_vector2
- ND_ifgreater_vector3
- ND_ifgreater_color3I
- ND_ifgreater_vector2I
- ND_ifgreater_vector3I
- ND_ifgreater_vector4I
- ND_ifgreatereq_float
- ND_ifgreatereq_color3
- ND_combine2_vector4VF
- ND_ifgreatereq_color4
- ND_ifgreatereq_floatI
- ND_dot_string
- ND_ifgreatereq_color4I
- ND_ifgreatereq_vector2I
- ND_ifequal_float
- ND_ifequal_color3
- ND_ifequal_color4
- ND_ifequal_vector3
- ND_ifequal_color3I
- ND_ifequal_vector3I
- ND_ifequal_vector4I
- ND_ifequal_color3B
- ND_ifequal_vector3B
- ND_ifequal_vector4B
- ND_switch_float
- ND_switch_color3
- ND_combine2_vector4VV
- ND_switch_color4
- ND_switch_vector4
- ND_switch_color3I
- ND_switch_color4I
- ND_switch_vector2I
- ND_switch_vector4I
- ND_convert_float_color3
- ND_convert_color3_color4
- ND_convert_float_vector3
- ND_convert_vector2_vector3
- ND_convert_boolean_surfaceshader
- ND_convert_vector3_color3
- ND_convert_vector3_vector2
- ND_convert_vector3_vector4
- ND_convert_vector4_color4
- ND_convert_color3_vector3
- ND_convert_color4_vector4
- ND_convert_color4_color3
- ND_convert_vector2_surfaceshader
- ND_convert_vector3_surfaceshader
- ND_convert_vector4_surfaceshader
- ND_convert_integer_surfaceshader
- ND_swizzle_float_vector2
- ND_swizzle_float_vector3
- ND_swizzle_color4_vector2
- ND_swizzle_float_vector4
- ND_swizzle_color3_color3
- ND_swizzle_color3_color4
- ND_swizzle_color3_vector3
- ND_swizzle_color3_vector4
- ND_swizzle_color4_color4
- ND_swizzle_color4_vector3
- ND_swizzle_color4_vector4
- ND_dot_boolean
- ND_swizzle_vector2_color3
- ND_swizzle_vector2_color4
- ND_swizzle_vector2_vector3
- ND_swizzle_vector3_float
- ND_swizzle_vector3_color4
- ND_swizzle_vector3_vector2
- ND_swizzle_vector3_vector3
- ND_swizzle_vector4_float
- ND_swizzle_vector4_color4
- ND_swizzle_vector4_vector2
- ND_swizzle_vector4_vector3
- ND_combine2_color4CF
- ND_combine3_color3
- ND_combine3_vector3
- ND_combine4_color4
- ND_combine4_vector4
- ND_extract_color3
- ND_extract_color4
- ND_separate2_vector2
- ND_separate4_vector4
- ND_blur_float
- ND_blur_color3
- ND_blur_color4
- ND_blur_vector2
- ND_heighttonormal_vector3
- ND_dot_color3
- ND_dot_color4
- ND_dot_vector2
- ND_dot_vector3
- ND_dot_matrix33
- ND_dot_matrix44
- ND_dot_filename
- ND_dot_volumeshader
- DiskLight
- PortalLight
- CylinderLight
- DomeLight
- SphereLight
- MeshLight
- VolumeLight
- UsdUVTexture
- UsdPrimvarReader_float
- UsdPrimvarReader_float3
- UsdPrimvarReader_int
- UsdPrimvarReader_normal
- UsdPrimvarReader_point
- UsdPrimvarReader_vector
- UsdPrimvarReader_matrix
```
</details>


## Referencing a MaterialX File from within USD

With MaterialX nodes registered it is possible to specify MaterialX nodes. One simple way to use MaterialX is to reference a MaterialX document from within a USD file. 

In the example below a basic geometry is created, a MaterialX document is referenced and a material is bound to the geometry.

```usd
#usda 1.0
(
    defaultPrim = "mxCapsule"
    upAxis = "Y"
    metersPerUnit = 0.01
)

def Capsule "mxCapsule" (
    prepend apiSchemas = ["MaterialBindingAPI"]
)
{
    rel material:binding = </MaterialX/Materials/Marble_3D>
}


def Scope "MaterialX" (
    references = @./gltf_sample_marble.mtlx@</MaterialX>
)
{
}
```

### "Flattening" The MaterialX Reference

To see the expanded contents of the reference file, `usdcat` can be used to "flatten" the reference using hte `-f` command option.

In [18]:
usd_with_materialXRef = './data/gltf_sample_marble_ref.usda'
usd_without_materialxRef = usd_with_materialXRef
usd_without_materialxRef = usd_without_materialxRef.replace('_ref.usda', '_no_ref.usda')
cmd = 'usdcat ' + usd_with_materialXRef + ' -f -o ' + usd_without_materialxRef + '> flatten.log 2>&1'
print('- Run: ' + '"' + cmd + '"')
result = os.system(cmd)
print('  - Flattening status: %s' % ('Success' if result == 0 else 'Failed'))
if result != 0:
    printFileRaw('flatten.log')

printFileAsXML(usd_without_materialxRef, 'Flattened Usd Content', 'usd')

- Run: "usdcat ./data/gltf_sample_marble_ref.usda -f -o ./data/gltf_sample_marble_no_ref.usda> flatten.log 2>&1"
  - Flattening status: Success


<details><summary>Flattened Usd Content</summary>

```usd
#usda 1.0
(
    defaultPrim = "mxCapsule"
    doc = """Generated from Composed Stage of root layer d:\\Work\\materialx\\MateralX_Learn_Private\\pymaterialx\\data\\gltf_sample_marble_ref.usda
"""
    metersPerUnit = 0.01
    upAxis = "Y"
)

def Capsule "mxCapsule" (
    apiSchemas = ["MaterialBindingAPI"]
)
{
    rel material:binding = </MaterialX/Materials/Marble_3D>
}

def Scope "MaterialX"
{
    def "Materials"
    {
        def Material "Marble_3D"
        {
            float inputs:alpha
            float inputs:alpha_cutoff
            int inputs:alpha_mode
            color3f inputs:attenuation_color
            float inputs:attenuation_distance
            color3f inputs:base_color
            float inputs:clearcoat
            float3 inputs:clearcoat_normal
            float inputs:clearcoat_roughness
            color3f inputs:emissive
            float inputs:emissive_strength
            float inputs:ior
            float inputs:iridescence
            float inputs:iridescence_ior
            float inputs:iridescence_thickness
            float inputs:metallic
            float3 inputs:normal
            float inputs:occlusion
            float inputs:roughness
            color3f inputs:sheen_color = (0, 0, 0)
            float inputs:sheen_roughness
            float inputs:specular
            color3f inputs:specular_color
            float3 inputs:tangent
            float inputs:thickness
            float inputs:transmission
            token outputs:mtlx:surface.connect = </MaterialX/Materials/Marble_3D/ND_gltf_pbr_surfaceshader.outputs:surface>

            def Shader "ND_gltf_pbr_surfaceshader"
            {
                uniform token info:id = "ND_gltf_pbr_surfaceshader"
                float inputs:alpha.connect = </MaterialX/Materials/Marble_3D.inputs:alpha>
                float inputs:alpha_cutoff.connect = </MaterialX/Materials/Marble_3D.inputs:alpha_cutoff>
                int inputs:alpha_mode.connect = </MaterialX/Materials/Marble_3D.inputs:alpha_mode>
                color3f inputs:attenuation_color.connect = </MaterialX/Materials/Marble_3D.inputs:attenuation_color>
                float inputs:attenuation_distance.connect = </MaterialX/Materials/Marble_3D.inputs:attenuation_distance>
                color3f inputs:base_color.connect = </MaterialX/Materials/Marble_3D/NG_marble1.outputs:out>
                float inputs:clearcoat.connect = </MaterialX/Materials/Marble_3D.inputs:clearcoat>
                float3 inputs:clearcoat_normal.connect = </MaterialX/Materials/Marble_3D.inputs:clearcoat_normal>
                float inputs:clearcoat_roughness.connect = </MaterialX/Materials/Marble_3D.inputs:clearcoat_roughness>
                color3f inputs:emissive.connect = </MaterialX/Materials/Marble_3D.inputs:emissive>
                float inputs:emissive_strength.connect = </MaterialX/Materials/Marble_3D.inputs:emissive_strength>
                float inputs:ior.connect = </MaterialX/Materials/Marble_3D.inputs:ior>
                float inputs:iridescence.connect = </MaterialX/Materials/Marble_3D.inputs:iridescence>
                float inputs:iridescence_ior.connect = </MaterialX/Materials/Marble_3D.inputs:iridescence_ior>
                float inputs:iridescence_thickness.connect = </MaterialX/Materials/Marble_3D.inputs:iridescence_thickness>
                float inputs:metallic.connect = </MaterialX/Materials/Marble_3D.inputs:metallic>
                float3 inputs:normal.connect = </MaterialX/Materials/Marble_3D.inputs:normal>
                float inputs:occlusion.connect = </MaterialX/Materials/Marble_3D.inputs:occlusion>
                float inputs:roughness.connect = </MaterialX/Materials/Marble_3D.inputs:roughness>
                color3f inputs:sheen_color.connect = </MaterialX/Materials/Marble_3D.inputs:sheen_color>
                float inputs:sheen_roughness.connect = </MaterialX/Materials/Marble_3D.inputs:sheen_roughness>
                float inputs:specular.connect = </MaterialX/Materials/Marble_3D.inputs:specular>
                color3f inputs:specular_color.connect = </MaterialX/Materials/Marble_3D.inputs:specular_color>
                float3 inputs:tangent.connect = </MaterialX/Materials/Marble_3D.inputs:tangent>
                float inputs:thickness.connect = </MaterialX/Materials/Marble_3D.inputs:thickness>
                float inputs:transmission.connect = </MaterialX/Materials/Marble_3D.inputs:transmission>
                token outputs:surface
            }

            def NodeGraph "NG_marble1"
            {
                color3f inputs:base_color_1 = (0.8, 0.8, 0.8) (
                    displayGroup = "Marble Color"
                    displayName = "Color 1"
                )
                color3f inputs:base_color_2 = (0.1, 0.1, 0.3) (
                    displayGroup = "Marble Color"
                    displayName = "Color 2"
                )
                int inputs:noise_octaves = 3 (
                    displayGroup = "Marble Noise"
                    displayName = "Octaves"
                )
                float inputs:noise_power = 3 (
                    displayGroup = "Marble Noise"
                    displayName = "Power"
                )
                float inputs:noise_scale_1 = 6 (
                    displayGroup = "Marble Noise"
                    displayName = "Scale 1"
                )
                float inputs:noise_scale_2 = 4 (
                    displayGroup = "Marble Noise"
                    displayName = "Scale 2"
                )
                color3f outputs:out.connect = </MaterialX/Materials/Marble_3D/NG_marble1/color_mix.outputs:out>

                def Shader "obj_pos"
                {
                    uniform token info:id = "ND_position_vector3"
                    float3 outputs:out
                }

                def Shader "add_xyz"
                {
                    uniform token info:id = "ND_dotproduct_vector3"
                    float3 inputs:in1.connect = </MaterialX/Materials/Marble_3D/NG_marble1/obj_pos.outputs:out>
                    float3 inputs:in2 = (1, 1, 1)
                    float outputs:out
                }

                def Shader "scale_xyz"
                {
                    uniform token info:id = "ND_multiply_float"
                    float inputs:in1.connect = </MaterialX/Materials/Marble_3D/NG_marble1/add_xyz.outputs:out>
                    float inputs:in2.connect = </MaterialX/Materials/Marble_3D/NG_marble1.inputs:noise_scale_1>
                    float outputs:out
                }

                def Shader "scale_pos"
                {
                    uniform token info:id = "ND_multiply_vector3FA"
                    float3 inputs:in1.connect = </MaterialX/Materials/Marble_3D/NG_marble1/obj_pos.outputs:out>
                    float inputs:in2.connect = </MaterialX/Materials/Marble_3D/NG_marble1.inputs:noise_scale_2>
                    float3 outputs:out
                }

                def Shader "noise"
                {
                    uniform token info:id = "ND_fractal3d_float"
                    int inputs:octaves.connect = </MaterialX/Materials/Marble_3D/NG_marble1.inputs:noise_octaves>
                    float3 inputs:position.connect = </MaterialX/Materials/Marble_3D/NG_marble1/scale_pos.outputs:out>
                    float outputs:out
                }

                def Shader "scale_noise"
                {
                    uniform token info:id = "ND_multiply_float"
                    float inputs:in1.connect = </MaterialX/Materials/Marble_3D/NG_marble1/noise.outputs:out>
                    float inputs:in2 = 3
                    float outputs:out
                }

                def Shader "sum"
                {
                    uniform token info:id = "ND_add_float"
                    float inputs:in1.connect = </MaterialX/Materials/Marble_3D/NG_marble1/scale_xyz.outputs:out>
                    float inputs:in2.connect = </MaterialX/Materials/Marble_3D/NG_marble1/scale_noise.outputs:out>
                    float outputs:out
                }

                def Shader "sin"
                {
                    uniform token info:id = "ND_sin_float"
                    float inputs:in.connect = </MaterialX/Materials/Marble_3D/NG_marble1/sum.outputs:out>
                    float outputs:out
                }

                def Shader "scale"
                {
                    uniform token info:id = "ND_multiply_float"
                    float inputs:in1.connect = </MaterialX/Materials/Marble_3D/NG_marble1/sin.outputs:out>
                    float inputs:in2 = 0.5
                    float outputs:out
                }

                def Shader "bias"
                {
                    uniform token info:id = "ND_add_float"
                    float inputs:in1.connect = </MaterialX/Materials/Marble_3D/NG_marble1/scale.outputs:out>
                    float inputs:in2 = 0.5
                    float outputs:out
                }

                def Shader "power"
                {
                    uniform token info:id = "ND_power_float"
                    float inputs:in1.connect = </MaterialX/Materials/Marble_3D/NG_marble1/bias.outputs:out>
                    float inputs:in2.connect = </MaterialX/Materials/Marble_3D/NG_marble1.inputs:noise_power>
                    float outputs:out
                }

                def Shader "color_mix"
                {
                    uniform token info:id = "ND_mix_color3"
                    color3f inputs:bg.connect = </MaterialX/Materials/Marble_3D/NG_marble1.inputs:base_color_1>
                    color3f inputs:fg.connect = </MaterialX/Materials/Marble_3D/NG_marble1.inputs:base_color_2>
                    float inputs:mix.connect = </MaterialX/Materials/Marble_3D/NG_marble1/power.outputs:out>
                    color3f outputs:out
                }
            }
        }
    }

    def "Shaders"
    {
        def Shader "ND_gltf_pbr_surfaceshader"
        {
            uniform token info:id = "ND_gltf_pbr_surfaceshader"
            token outputs:surface
        }
    }

    def "NodeGraphs"
    {
        def NodeGraph "NG_marble1"
        {
            color3f inputs:base_color_1 = (0.8, 0.8, 0.8) (
                displayGroup = "Marble Color"
                displayName = "Color 1"
            )
            color3f inputs:base_color_2 = (0.1, 0.1, 0.3) (
                displayGroup = "Marble Color"
                displayName = "Color 2"
            )
            int inputs:noise_octaves = 3 (
                displayGroup = "Marble Noise"
                displayName = "Octaves"
            )
            float inputs:noise_power = 3 (
                displayGroup = "Marble Noise"
                displayName = "Power"
            )
            float inputs:noise_scale_1 = 6 (
                displayGroup = "Marble Noise"
                displayName = "Scale 1"
            )
            float inputs:noise_scale_2 = 4 (
                displayGroup = "Marble Noise"
                displayName = "Scale 2"
            )
            color3f outputs:out.connect = </MaterialX/NodeGraphs/NG_marble1/color_mix.outputs:out>

            def Shader "obj_pos"
            {
                uniform token info:id = "ND_position_vector3"
                float3 outputs:out
            }

            def Shader "add_xyz"
            {
                uniform token info:id = "ND_dotproduct_vector3"
                float3 inputs:in1.connect = </MaterialX/NodeGraphs/NG_marble1/obj_pos.outputs:out>
                float3 inputs:in2 = (1, 1, 1)
                float outputs:out
            }

            def Shader "scale_xyz"
            {
                uniform token info:id = "ND_multiply_float"
                float inputs:in1.connect = </MaterialX/NodeGraphs/NG_marble1/add_xyz.outputs:out>
                float inputs:in2.connect = </MaterialX/NodeGraphs/NG_marble1.inputs:noise_scale_1>
                float outputs:out
            }

            def Shader "scale_pos"
            {
                uniform token info:id = "ND_multiply_vector3FA"
                float3 inputs:in1.connect = </MaterialX/NodeGraphs/NG_marble1/obj_pos.outputs:out>
                float inputs:in2.connect = </MaterialX/NodeGraphs/NG_marble1.inputs:noise_scale_2>
                float3 outputs:out
            }

            def Shader "noise"
            {
                uniform token info:id = "ND_fractal3d_float"
                int inputs:octaves.connect = </MaterialX/NodeGraphs/NG_marble1.inputs:noise_octaves>
                float3 inputs:position.connect = </MaterialX/NodeGraphs/NG_marble1/scale_pos.outputs:out>
                float outputs:out
            }

            def Shader "scale_noise"
            {
                uniform token info:id = "ND_multiply_float"
                float inputs:in1.connect = </MaterialX/NodeGraphs/NG_marble1/noise.outputs:out>
                float inputs:in2 = 3
                float outputs:out
            }

            def Shader "sum"
            {
                uniform token info:id = "ND_add_float"
                float inputs:in1.connect = </MaterialX/NodeGraphs/NG_marble1/scale_xyz.outputs:out>
                float inputs:in2.connect = </MaterialX/NodeGraphs/NG_marble1/scale_noise.outputs:out>
                float outputs:out
            }

            def Shader "sin"
            {
                uniform token info:id = "ND_sin_float"
                float inputs:in.connect = </MaterialX/NodeGraphs/NG_marble1/sum.outputs:out>
                float outputs:out
            }

            def Shader "scale"
            {
                uniform token info:id = "ND_multiply_float"
                float inputs:in1.connect = </MaterialX/NodeGraphs/NG_marble1/sin.outputs:out>
                float inputs:in2 = 0.5
                float outputs:out
            }

            def Shader "bias"
            {
                uniform token info:id = "ND_add_float"
                float inputs:in1.connect = </MaterialX/NodeGraphs/NG_marble1/scale.outputs:out>
                float inputs:in2 = 0.5
                float outputs:out
            }

            def Shader "power"
            {
                uniform token info:id = "ND_power_float"
                float inputs:in1.connect = </MaterialX/NodeGraphs/NG_marble1/bias.outputs:out>
                float inputs:in2.connect = </MaterialX/NodeGraphs/NG_marble1.inputs:noise_power>
                float outputs:out
            }

            def Shader "color_mix"
            {
                uniform token info:id = "ND_mix_color3"
                color3f inputs:bg.connect = </MaterialX/NodeGraphs/NG_marble1.inputs:base_color_1>
                color3f inputs:fg.connect = </MaterialX/NodeGraphs/NG_marble1.inputs:base_color_2>
                float inputs:mix.connect = </MaterialX/NodeGraphs/NG_marble1/power.outputs:out>
                color3f outputs:out
            }
        }
    }
}

```
</details>


To test that the file is valid, `usdview` can be used to examine the file as shown below.
<image src="../documents/images/gltf_sample_ref_flattened_usdview.png" width="800px">

### UsdMtlx API

Another way to load in MaterialX is via the `UsdMtlx` module.

At time of writing there the exposed API entry points in Python are:
MaterialX input from file or string: 
  - `TestFile()`: To import from a MaterialX file.
  - `TestString()`: To import from a MaterialX string (XML). 

There are no APIs in this module to export to MaterialX at the current time.

> Note that a utility function was written to discover the module functions as the test `inspect.isfunction` does not seem to detect detect Boost Python function members. ( Currently `Boost`` is used to expose the C++ API to Python. )

In [19]:
from pxr import UsdMtlx
import inspect

def is_function(obj):
    if hasattr(obj, "__class__"):
        cl = getattr(obj, '__class__')
        if obj.__class__.__name__ == "builtin_function_or_method":
            return True    
    return False

members = inspect.getmembers(UsdMtlx)

# Filter the members to find Boost functions.
usdMtlx_functions = [member for member in members if is_function(member[1])]

# Print the functions found.
for name, function in usdMtlx_functions:
    print(f"- UsdMtlx function: {name}")


- UsdMtlx function: _TestFile
- UsdMtlx function: _TestString


These utilities are sufficient to perform conversion from MaterialX to a USD stage. 

The stage can then be modified / saved as desired. 

In the example below we take the same MaterialX file, load it into a stage and write it to a file. 

Note that:
1. The MaterialX file is is not a referenced. 
2. There appears no way to import a MaterialX document into an existing stage. 
2. Additional meta data is added which stored the MaterialX documents rendering color space. (At time of writing input color spaces are not supported.)

To create the equivalent of the USD file above, the appropriate geometry and binding can be added to the stage.

In [20]:
# Read in a sample MaterialX file into a new stage
stage = UsdMtlx._TestFile('./data/gltf_sample_marble.mtlx')
#colorspace = stage.GetColorSpace()

# Export the stage to disk and also output for display.
exportedUsd = './data/gltf_sample_marble_usdmtlx.usda'
exported = stage.GetRootLayer().Export(exportedUsd)
if exported:
    print('Converted MaterialX to Usd file:', exportedUsd)

stringResult = stage.GetRootLayer().ExportToString()
text = '<details><summary>UsdMtlx Conversion Results</summary>\n\n' + '```usd\n' + stringResult + '```\n' + '</details>\n' 
display_markdown(text , raw=True)


Converted MaterialX to Usd file: ./data/gltf_sample_marble_usdmtlx.usda


<details><summary>UsdMtlx Conversion Results</summary>

```usd
#usda 1.0
(
    customLayerData = {
        string colorSpace = "lin_rec709"
    }
)

def "MaterialX"
{
    def "Materials"
    {
        def Material "Marble_3D"
        {
            float inputs:alpha
            float inputs:alpha_cutoff
            int inputs:alpha_mode
            color3f inputs:attenuation_color
            float inputs:attenuation_distance
            color3f inputs:base_color
            float inputs:clearcoat
            float3 inputs:clearcoat_normal
            float inputs:clearcoat_roughness
            color3f inputs:emissive
            float inputs:emissive_strength
            float inputs:ior
            float inputs:iridescence
            float inputs:iridescence_ior
            float inputs:iridescence_thickness
            float inputs:metallic
            float3 inputs:normal
            float inputs:occlusion
            float inputs:roughness
            color3f inputs:sheen_color = (0, 0, 0)
            float inputs:sheen_roughness
            float inputs:specular
            color3f inputs:specular_color
            float3 inputs:tangent
            float inputs:thickness
            float inputs:transmission
            token outputs:mtlx:surface.connect = </MaterialX/Materials/Marble_3D/ND_gltf_pbr_surfaceshader.outputs:surface>

            def Shader "ND_gltf_pbr_surfaceshader" (
                prepend references = </MaterialX/Shaders/ND_gltf_pbr_surfaceshader>
            )
            {
                float inputs:alpha.connect = </MaterialX/Materials/Marble_3D.inputs:alpha>
                float inputs:alpha_cutoff.connect = </MaterialX/Materials/Marble_3D.inputs:alpha_cutoff>
                int inputs:alpha_mode.connect = </MaterialX/Materials/Marble_3D.inputs:alpha_mode>
                color3f inputs:attenuation_color.connect = </MaterialX/Materials/Marble_3D.inputs:attenuation_color>
                float inputs:attenuation_distance.connect = </MaterialX/Materials/Marble_3D.inputs:attenuation_distance>
                color3f inputs:base_color.connect = </MaterialX/Materials/Marble_3D/NG_marble1.outputs:out>
                float inputs:clearcoat.connect = </MaterialX/Materials/Marble_3D.inputs:clearcoat>
                float3 inputs:clearcoat_normal.connect = </MaterialX/Materials/Marble_3D.inputs:clearcoat_normal>
                float inputs:clearcoat_roughness.connect = </MaterialX/Materials/Marble_3D.inputs:clearcoat_roughness>
                color3f inputs:emissive.connect = </MaterialX/Materials/Marble_3D.inputs:emissive>
                float inputs:emissive_strength.connect = </MaterialX/Materials/Marble_3D.inputs:emissive_strength>
                float inputs:ior.connect = </MaterialX/Materials/Marble_3D.inputs:ior>
                float inputs:iridescence.connect = </MaterialX/Materials/Marble_3D.inputs:iridescence>
                float inputs:iridescence_ior.connect = </MaterialX/Materials/Marble_3D.inputs:iridescence_ior>
                float inputs:iridescence_thickness.connect = </MaterialX/Materials/Marble_3D.inputs:iridescence_thickness>
                float inputs:metallic.connect = </MaterialX/Materials/Marble_3D.inputs:metallic>
                float3 inputs:normal.connect = </MaterialX/Materials/Marble_3D.inputs:normal>
                float inputs:occlusion.connect = </MaterialX/Materials/Marble_3D.inputs:occlusion>
                float inputs:roughness.connect = </MaterialX/Materials/Marble_3D.inputs:roughness>
                color3f inputs:sheen_color.connect = </MaterialX/Materials/Marble_3D.inputs:sheen_color>
                float inputs:sheen_roughness.connect = </MaterialX/Materials/Marble_3D.inputs:sheen_roughness>
                float inputs:specular.connect = </MaterialX/Materials/Marble_3D.inputs:specular>
                color3f inputs:specular_color.connect = </MaterialX/Materials/Marble_3D.inputs:specular_color>
                float3 inputs:tangent.connect = </MaterialX/Materials/Marble_3D.inputs:tangent>
                float inputs:thickness.connect = </MaterialX/Materials/Marble_3D.inputs:thickness>
                float inputs:transmission.connect = </MaterialX/Materials/Marble_3D.inputs:transmission>
            }

            def "NG_marble1" (
                prepend references = </MaterialX/NodeGraphs/NG_marble1>
            )
            {
            }
        }
    }

    def "Shaders"
    {
        def Shader "ND_gltf_pbr_surfaceshader"
        {
            uniform token info:id = "ND_gltf_pbr_surfaceshader"
            token outputs:surface
        }
    }

    def "NodeGraphs"
    {
        def NodeGraph "NG_marble1"
        {
            color3f inputs:base_color_1 = (0.8, 0.8, 0.8) (
                displayGroup = "Marble Color"
                displayName = "Color 1"
            )
            color3f inputs:base_color_2 = (0.1, 0.1, 0.3) (
                displayGroup = "Marble Color"
                displayName = "Color 2"
            )
            int inputs:noise_octaves = 3 (
                displayGroup = "Marble Noise"
                displayName = "Octaves"
            )
            float inputs:noise_power = 3 (
                displayGroup = "Marble Noise"
                displayName = "Power"
            )
            float inputs:noise_scale_1 = 6 (
                displayGroup = "Marble Noise"
                displayName = "Scale 1"
            )
            float inputs:noise_scale_2 = 4 (
                displayGroup = "Marble Noise"
                displayName = "Scale 2"
            )
            color3f outputs:out.connect = </MaterialX/NodeGraphs/NG_marble1/color_mix.outputs:out>

            def Shader "obj_pos"
            {
                uniform token info:id = "ND_position_vector3"
                float3 outputs:out
            }

            def Shader "add_xyz"
            {
                uniform token info:id = "ND_dotproduct_vector3"
                float3 inputs:in1.connect = </MaterialX/NodeGraphs/NG_marble1/obj_pos.outputs:out>
                float3 inputs:in2 = (1, 1, 1)
                float outputs:out
            }

            def Shader "scale_xyz"
            {
                uniform token info:id = "ND_multiply_float"
                float inputs:in1.connect = </MaterialX/NodeGraphs/NG_marble1/add_xyz.outputs:out>
                float inputs:in2.connect = </MaterialX/NodeGraphs/NG_marble1.inputs:noise_scale_1>
                float outputs:out
            }

            def Shader "scale_pos"
            {
                uniform token info:id = "ND_multiply_vector3FA"
                float3 inputs:in1.connect = </MaterialX/NodeGraphs/NG_marble1/obj_pos.outputs:out>
                float inputs:in2.connect = </MaterialX/NodeGraphs/NG_marble1.inputs:noise_scale_2>
                float3 outputs:out
            }

            def Shader "noise"
            {
                uniform token info:id = "ND_fractal3d_float"
                int inputs:octaves.connect = </MaterialX/NodeGraphs/NG_marble1.inputs:noise_octaves>
                float3 inputs:position.connect = </MaterialX/NodeGraphs/NG_marble1/scale_pos.outputs:out>
                float outputs:out
            }

            def Shader "scale_noise"
            {
                uniform token info:id = "ND_multiply_float"
                float inputs:in1.connect = </MaterialX/NodeGraphs/NG_marble1/noise.outputs:out>
                float inputs:in2 = 3
                float outputs:out
            }

            def Shader "sum"
            {
                uniform token info:id = "ND_add_float"
                float inputs:in1.connect = </MaterialX/NodeGraphs/NG_marble1/scale_xyz.outputs:out>
                float inputs:in2.connect = </MaterialX/NodeGraphs/NG_marble1/scale_noise.outputs:out>
                float outputs:out
            }

            def Shader "sin"
            {
                uniform token info:id = "ND_sin_float"
                float inputs:in.connect = </MaterialX/NodeGraphs/NG_marble1/sum.outputs:out>
                float outputs:out
            }

            def Shader "scale"
            {
                uniform token info:id = "ND_multiply_float"
                float inputs:in1.connect = </MaterialX/NodeGraphs/NG_marble1/sin.outputs:out>
                float inputs:in2 = 0.5
                float outputs:out
            }

            def Shader "bias"
            {
                uniform token info:id = "ND_add_float"
                float inputs:in1.connect = </MaterialX/NodeGraphs/NG_marble1/scale.outputs:out>
                float inputs:in2 = 0.5
                float outputs:out
            }

            def Shader "power"
            {
                uniform token info:id = "ND_power_float"
                float inputs:in1.connect = </MaterialX/NodeGraphs/NG_marble1/bias.outputs:out>
                float inputs:in2.connect = </MaterialX/NodeGraphs/NG_marble1.inputs:noise_power>
                float outputs:out
            }

            def Shader "color_mix"
            {
                uniform token info:id = "ND_mix_color3"
                color3f inputs:bg.connect = </MaterialX/NodeGraphs/NG_marble1.inputs:base_color_1>
                color3f inputs:fg.connect = </MaterialX/NodeGraphs/NG_marble1.inputs:base_color_2>
                float inputs:mix.connect = </MaterialX/NodeGraphs/NG_marble1/power.outputs:out>
                color3f outputs:out
            }
        }
    }
}

```
</details>
